In [1]:
import pandas as pd


df = pd.read_csv("test.csv")
better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', '01', 'V1', 'V2', 'V3', 'V4', 'F1', 'F2', 'F3', 'F4', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'A1', 'A2', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'P1', 'P2', 'P3', 'C1', 'C2', 'C3', 'C4', 'C5']
df.columns = better_cols

## Make a better index column and dump everything not grade-related
df['Name'] = df['Last Name'] + ', ' + df['First Name']
df.drop(['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability'], axis=1, inplace=True)
df.set_index("Name", inplace=True)


In [2]:
## Define the tools for reading the strings into numeric values: 

def sgn_to_number(sgn):
#Note that S has the value 1, G has the value 0.5, and N has the value 0; else is 0; 
    if (sgn == "S"):
        return 1
    elif (sgn == "G"):
        return 0.5
    else:
        return 0
    
def marks_to_score(marks):
    ## First, extract the last two entries:
    try: 
        marks = marks[-2:]
    except TypeError:  # encountered NaN or not a string
        return 0
    except IndexError: # encountered NaN; bah
        return 0
    
    return {"score":sum(map(sgn_to_number, marks)), "number of ns": marks.count("N")+marks.count("Z")}
    
    
## Make sure nothing broke 
assert(marks_to_score("SSNSGS") == {"score":1.5, "number of ns":0})

In [3]:
def summative(marks):
    score_so_far = 0
    ns_so_far = 0
    for cell in marks:
        try: 
            score_so_far += cell['score']
            ns_so_far += cell['number of ns']
        except:
            pass
    return {'Score':score_so_far, 'Number of ns': ns_so_far}

In [52]:
class_scores = list()
for index, row in df.iterrows():
    raw = pd.DataFrame({'Standard':row.index, 'Marks':row.values})
    raw = raw.dropna()
    class_scores.append( {"Name":index, "Marks":summative(row.apply(marks_to_score)), "Raw Data":raw})

## Dataset for this is a hashset
## Key      |   Data
## ---------+---------
## Name     |  full name:  last, first
## Marks    |  hash of {'Score' and 'Number of ns'}
## Raw Data |  full pandas Series whose index is the Name above

In [65]:
# Now let's export the PDF report:
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
from datetime import date

today = date.today().strftime("%B %d, %Y")

env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("template.html")
template_vars = {
    "Name": class_scores[15]['Name'],
    "raw_data": class_scores[15]['Raw Data'].to_html(index=False, bold_rows=False, classes=["paleGrayRows"]),
    "score": class_scores[15]['Marks']['Score'],
    "number_of_ns": class_scores[15]['Marks']['Number of ns'],
    "letter_grade": "QQQQQQQQQQ",
    "date": today,
    "midterm_or_final": "Midterm"
}

output = template.render(template_vars)
#print(output)
HTML(string=output).write_pdf("test.pdf",stylesheets=["./report.css"])